In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from data_preproduction import load_data
import pickle

In [46]:
def random_sample(path="./res/fragrantica_filtered_with_description.csv", size=1000):
    df = load_data(path)
    df_sample, _ = train_test_split(df, train_size=size, stratify=df['Brand'], random_state=42)
    return df_sample

def corpus():
    df2 = pd.read_csv("./res/pda_samples.csv")
    sample_size = 1000  # 원하는 샘플 개수
    df1_sample = random_sample(sample_size)
    df1_corpus = df1_sample['olfactory'].tolist()
    df2_corpus = df2['description'].tolist()
    corpus = df1_corpus + df2_corpus
    with open("./res/corpus.pkl","wb") as f:
        pickle.dump(corpus, f)
    return corpus

In [40]:
with open("./res/corpus.pkl","rb") as f:
    corpus = pickle.load(f)

In [41]:
len(corpus)

1062

In [51]:
df_sample = random_sample(size=651)

In [52]:
df_sample.head()

,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5,description,olfactory
5596,https://www.fragrantica.com/perfume/eau-d-ital...,eau d italie,eau d italie,Italy,unisex,3.86,372,2004,"bergamot, black currant, incense","clay, magnolia, tuberose","clover, musk, patchouli, amber",bertrand duchaufour,NaN,floral,clay,green,citrus,earthy,Eau D Italie by Eau D Italie is a unisex fragr...,Eau D Italie by Eau D Italie is a unisex fragr...
10024,https://www.fragrantica.com/perfume/brecourt/r...,rosa gallica,brecourt,France,unisex,4.04,132,2012,"incense, pink pepper","rose, myrrh","ebony tree, cashmere wood, ambergris",emilie bouge,NaN,rose,amber,woody,balsamic,warm spicy,Rosa Gallica by Brecourt is a unisex fragrance...,Rosa Gallica by Brecourt is a unisex fragrance...
1728,https://www.fragrantica.com/perfume/jo-malone-...,rose blush cologne 2023,jo malone london,UK,women,3.65,74,2023,"litchi, basil",rose,white musk,NaN,NaN,rose,musky,fruity,tropical,floral,Rose Blush Cologne 2023 by Jo Malone London is...,Rose Blush Cologne 2023 by Jo Malone London is...
13244,https://www.fragrantica.com/perfume/lattafa-pe...,qimmah for women,lattafa perfumes,UAE,women,4.20,284,2022,"almond, coffee","tuberose, jasmine, tonka","vanilla, cacao, sandalwood",NaN,NaN,vanilla,white floral,warm spicy,sweet,almond,Qimmah For Women by Lattafa Perfumes is a wome...,Qimmah For Women by Lattafa Perfumes is a wome...
8271,https://www.fragrantica.com/perfume/santa-mari...,alba di seoul,santa maria novella,Italy,unisex,3.97,117,2012,green notes,pine tree,"woody notes, oriental notes",NaN,NaN,woody,green,aromatic,warm spicy,amber,Alba Di Seoul by Santa Maria Novella is a unis...,Alba Di Seoul by Santa Maria Novella is a unis...


In [53]:
import numpy as np
import os
from dotenv import load_dotenv
from openai import OpenAI


load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai_client = OpenAI(api_key=OPENAI_API_KEY)
model_name = "gpt-4o-mini"

def chatgpt_generate(query):
    messages = [{
        "role": "system",
        "content" : "You are a helpful assistant."},
        {
            "role": "user",
            "content": query
        }]
    response = openai_client.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    return answer

In [ ]:
i = 2
prompt = 

In [ ]:
prompt_template = f"""You are a fragrance expert tasked with summarizing perfume descriptions based on\
given data and additional information from the Fragrantica website. \
Your task is to generate a concise and engaging description of each perfume, \
focusing on its mood and user reviews. Follow these guidelines:\
1. **Content**:
   - Focus on the overall mood, olfactory characteristics and users assessment of the perfume.\
   - Summarize user reviews to highlight key impressions (e.g., "elegant," "fresh," "long-lasting").\
   - You can find fundamental features of the perfume from the given description:{df_sample['description'][i]} but 
   do not directly copy sentences from that.\
   - You have to retrive user review from the website url:{df_sample['url'][i]}.
   - Do not explicitly mention the Top, Middle, or Base notes, but imply them through descriptive language\
   (because your description will be used for fine tuning model which predicts olfactory notes from the description).
2. **Tone**: Use professional yet engaging language suitable for a fragrance description.
3. **Example**:
   - "The first new feminine Creed fragrance for five years (the last was 2000 Fleurs), and the first to be debuted in the USA.
    The scent is inspired by Olivier Creed's travels on the high seas, 'an element of nature that connects all humanity' and according to Creed, 
    the bottle evokes the shoulders of a feminine figure as she rests upon white sands caressed by the ocean's gentle current. 
    A silver ribbon at the neck conveys sunshine dancing on the Aegean sea."
    """